In [ ]:
from dask.distributed import Client, wait
from dask_kubernetes import KubeCluster
cluster = KubeCluster.from_yaml('/home/krughoff/dask/dask_worker.yml')

In [ ]:
c = cluster.scale_up(100)

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
from dask import dataframe as dd

In [ ]:
df = dd.read_parquet('/project/shared/data/gaia_dr2_1am/part-*', columns=['l', 'b'], engine='pyarrow')

In [ ]:
len(df)

In [ ]:
df = client.persist(df)

In [ ]:
len(df)

In [ ]:
import holoviews as hv
import holoviews.operation.datashader as hd
hd.shade.cmap=["lightblue", "darkblue"]
hv.extension("bokeh", "matplotlib")

In [ ]:
points = hv.Points(df)

In [ ]:
%%opts RGB [width=1000, height=500]
hd.datashade(points)

In [ ]:
c.close()